# Laporan Proyek Machine Learning - Leo Prangs Tobing

In [128]:
# from google.colab import drive
# drive.mount('/content/drive')

## Project Overview

**Latar Belakang:**

Steam adalah salah satu platform distribusi game digital terbesar di dunia dengan jutaan pengguna aktif. Dengan jumlah game yang terus bertambah setiap tahun, pengguna sering mengalami kesulitan dalam menemukan game yang sesuai dengan preferensi mereka. Hal ini menciptakan kebutuhan akan sistem rekomendasi yang cerdas untuk membantu pengguna menavigasi katalog game yang luas.

Sistem rekomendasi telah menjadi komponen penting dalam berbagai platform digital untuk meningkatkan personalisasi dan pengalaman pengguna. Dalam konteks Steam, sistem ini dapat membantu pengguna menemukan game yang relevan berdasarkan minat atau riwayat interaksi mereka. Selain itu, sistem rekomendasi juga berperan dalam meningkatkan keterlibatan pengguna dan konversi pembelian, yang pada akhirnya berdampak pada keuntungan platform secara keseluruhan.

Beberapa pendekatan umum dalam sistem rekomendasi meliputi **Content-Based Filtering** dan **Collaborative Filtering**. Content-Based Filtering bekerja dengan membandingkan fitur konten antar item (misalnya genre, tag, developer), sedangkan Collaborative Filtering merekomendasikan item berdasarkan perilaku pengguna lain yang memiliki preferensi serupa. Menurut Aggarwal (2016), kedua pendekatan ini merupakan fondasi dari kebanyakan sistem rekomendasi modern, dan masing-masing memiliki keunggulan serta keterbatasan tergantung pada konteks penggunaannya. Di sisi lain, Jannach et al. (2010) juga menekankan pentingnya sistem rekomendasi dalam meningkatkan pengalaman pengguna dan efisiensi pencarian, terutama dalam lingkungan dengan pilihan item yang sangat banyak seperti pada toko game digital.

**Mengapa masalah ini penting:**

- Membantu pengguna menemukan game yang sesuai dengan minat mereka (Jannach et al., 2010).
- Meningkatkan konversi pembelian game di platform (Aggarwal, 2016).
- Mendorong penggunaan platform secara lebih intensif dan mempertahankan pengguna aktif (Aggarwal, 2016; Jannach et al., 2010).

**Referensi:**

- Aggarwal, C. C. (2016). *Recommender Systems: The Textbook*. Springer.
- Jannach, D., Adomavicius, G., Tuzhilin, A., & Kantor, P. (2010). *Recommender Systems – Challenges, Insights and Research Opportunities*. ACM Transactions on Intelligent Systems and Technology (TIST), 1(1), 1–38.

## Business Understanding

**1. Problem Statements**

- Bagaimana merekomendasikan game kepada pengguna baru berdasarkan konten game (genre, tag, developer)?
- Bagaimana memberikan rekomendasi personal kepada pengguna berdasarkan kesamaan rating/ulasan dengan pengguna lain?

**2. Goals**

- Membangun sistem rekomendasi berbasis konten (Content-Based Filtering) untuk memberikan rekomendasi berdasarkan fitur game.
- Membangun sistem Collaborative Filtering berbasis user-rating/interaction.
- Membandingkan performa kedua metode.

**3. Solution Statements**

- **Solution 1: Content-Based Filtering**
  Menggunakan fitur seperti genre, tag, dan developer untuk menghitung kemiripan antar game menggunakan TF-IDF + cosine similarity.

- **Solution 2: Collaborative Filtering**
  Menggunakan Matrix Factorization (SVD) berdasarkan data interaksi pengguna (misal: review, rating atau playtime) untuk memprediksi preferensi pengguna.


## Data Understanding

### Load Data

Dataset yang digunakan adalah **Steam Store Games** dari [Kaggle](https://www.kaggle.com/datasets/nikdavis/steam-store-games), berisi informasi lengkap tentang game di platform Steam, termasuk konten, popularitas, dan metadata lainnya.

In [129]:
# !pip install numpy==1.24.4 --force-reinstall
# !pip install scikit-surprise==1.1.4 --force-reinstall

In [ ]:
# load library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Preprocesasing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import numpy as np # ajust dtype
from sklearn.preprocessing import MinMaxScaler

# modelling
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# evaluation
from collections import defaultdict

# Untuk .py file (run in terminal)
from IPython.display import display

# df = pd.read_csv("/content/drive/MyDrive/Dicoding/steam.csv")
df = pd.read_csv("steam.csv")
display(df.head())
df.shape

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat enabled,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


(27075, 18)

**Insight:**

terdapat **27.075 baris** data:

| Fitur              | Deskripsi                                                  |
| ------------------ | ---------------------------------------------------------- |
| `appid`            | ID unik dari game                                          |
| `name`             | Nama game                                                  |
| `release_date`     | Tanggal rilis game                                         |
| `english`          | Boolean (1 jika game berbahasa Inggris)                    |
| `developer`        | Nama pengembang game                                       |
| `publisher`        | Nama penerbit game                                         |
| `platforms`        | Platform tempat game dapat dimainkan (Windows, Linux, Mac) |
| `required_age`     | Umur minimal pemain                                        |
| `categories`       | Kategori game seperti Singleplayer, Multiplayer, Co-op     |
| `genres`           | Genre game seperti Action, Strategy, Adventure             |
| `steamspy_tags`    | Tag deskriptif yang diberikan oleh komunitas Steam         |
| `achievements`     | Jumlah pencapaian dalam game                               |
| `positive_ratings` | Jumlah ulasan positif dari pengguna                        |
| `negative_ratings` | Jumlah ulasan negatif dari pengguna                        |
| `average_playtime` | Rata-rata waktu bermain (dalam menit)                      |
| `price`            | Harga game (dalam USD)                                     |
| `initialprice`     | Harga awal sebelum diskon                                  |
| `discount`         | Persentase diskon game (jika ada)                          |
| `rating`           | Rating game (jika tersedia)                                |

### EDA
Beberapa analisis penting

In [131]:
display(df.info()) # release_date to datetime

print('df.isna().sum():\n',df.isna().sum()) # developer 1, publisher 14

dupe = df['name'][df['name'].duplicated()].unique()
print(f"duplicated name ({len(dupe)}): {dupe}") # 41 nama, wajar karena public domain

display('df.describe().T: ',df.describe().T) # price 75% < 7.19, untuk range 0 - 421.99

for col in ['genres', 'steamspy_tags', 'categories', 'developer']:
    print(f"{col}: {df[col].unique().tolist()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27074 non-null  object 
 5   publisher         27061 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

None

df.isna().sum():
 appid                0
name                 0
release_date         0
english              0
developer            1
publisher           14
platforms            0
required_age         0
categories           0
genres               0
steamspy_tags        0
achievements         0
positive_ratings     0
negative_ratings     0
average_playtime     0
median_playtime      0
owners               0
price                0
dtype: int64
duplicated name (41): ['Dark Matter' 'Ultimate Arena' 'New York Bus Simulator' 'Taxi'
 'Mars 2030' 'Rumpus' 'Invasion' 'Alter Ego' 'Castles' 'Luna' 'Slice&Dice'
 'Killing Time' 'The Tower' 'Colony' 'Beyond the Wall' 'Solitaire'
 'Escape Room' 'Escape' 'Dodge' 'Mystical' 'The Great Escape' 'Bounce'
 'City Builder' 'RUSH' 'Evolution' 'Alone' 'Scorch' 'The Mine' 'Ashes'
 'Exodus' 'Space Maze' 'Nightmare Simulator' 'Fireflies' 'Surge'
 "Santa's Workshop" 'Chaos Theory' 'Zombie Apocalypse' 'Cortex'
 'Hide and Seek' 'Experience' '2048']


'df.describe().T: '

,count,mean,std,min,25%,50%,75%,max
appid,27075.0,596203.508661,250894.167246,10.0,401230.00,599070.00,798760.00,1069460.00
english,27075.0,0.981127,0.136081,0.0,1.00,1.00,1.00,1.00
required_age,27075.0,0.354903,2.406044,0.0,0.00,0.00,0.00,18.00
achievements,27075.0,45.248864,352.670281,0.0,0.00,7.00,23.00,9821.00
positive_ratings,27075.0,1000.558523,18988.724439,0.0,6.00,24.00,126.00,2644404.00
negative_ratings,27075.0,211.027147,4284.938531,0.0,2.00,9.00,42.00,487076.00
average_playtime,27075.0,149.804949,1827.038141,0.0,0.00,0.00,0.00,190625.00
median_playtime,27075.0,146.056030,2353.880080,0.0,0.00,0.00,0.00,190625.00
price,27075.0,6.078193,7.874922,0.0,1.69,3.99,7.19,421.99


genres: ['Action', 'Action;Free to Play', 'Action;Free to Play;Strategy', 'Action;Adventure', 'Indie', 'Indie;Strategy', 'Strategy', 'RPG', 'Animation & Modeling;Video Production', 'Action;RPG', 'Action;Indie;RPG', 'Casual;Indie', 'Action;Indie;Strategy', 'Action;Simulation;Strategy', 'Simulation;Strategy', 'Action;Simulation', 'Racing', 'RPG;Strategy', 'Adventure;Casual', 'Casual', 'Strategy;Casual', 'Action;Casual', 'Indie;Simulation', 'Casual;Strategy;Indie', 'Action;Indie', 'Action;Strategy', 'Violent;Action;Strategy', 'Adventure', 'Action;Adventure;Racing;Simulation;Strategy', 'Adventure;RPG', 'Action;Adventure;RPG', 'Action;Free to Play;Massively Multiplayer', 'Indie;Simulation;Strategy', 'Nudity;Violent;Action;Adventure;RPG', 'Nudity;Violent;RPG', 'Action;Free to Play;Massively Multiplayer;RPG;Strategy', 'Racing;Simulation;Sports', 'Free to Play;Massively Multiplayer;RPG', 'Sports', 'Simulation', 'Casual;Indie;Racing', 'Action;Casual;Indie', 'RPG;Casual;Adventure', 'Action;Adven

**Insight:**
- **change dtype:** `release_date` to datetime
- **null value:** `developer` = 1, `publisher` = 14
- **duplicate:** `name` = 41, wajar karena public domain
- **number distribution:** `price` 75% is below 7.19, untuk range 0 - 421.99
- **feature**: `genres` & `steamspy_tags` memiliki data yang mirip

## Data Preparation

### Clean DF (df2)

perbaikan df: null dan duplikasi nama. `df2` yang akan dipakai dalam evaluasi

In [132]:
df2 = df.copy(deep=True)

# mengisi nilai kosong pada developer dan publisher
df2['developer'] = df2['developer'].fillna('')
df2['publisher'] = df2['publisher'].fillna('')

# ubah name jadi unik karena rekomendasi menggunakan nama sebagai index
df2['name'] = df2['name'].str.lower()
df2 = df2.drop_duplicates(subset='name', keep='first')
df2.sample(5)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
12507,569770,starship constructor,2017-07-19,1,SSCTEAM,SSCTEAM,windows,0,Single-player;Steam Achievements;Steam Cloud,Indie;Early Access,Early Access;Indie;Space,15,150,51,0,0,20000-50000,3.99
9864,497940,destiny of ancient kingdoms™,2016-08-29,1,Utopia Dream Entertainment Alliance (Pty) Ltd - South Africa,Utopia Dream Entertainment Alliance (Pty) Ltd - South Africa,windows,0,Multi-player;Online Multi-Player;MMO;Steam Trading Cards;In-App Purchases,Action;Adventure;Free to Play;Massively Multiplayer;RPG;Strategy,Free to Play;Massively Multiplayer;RPG,0,80,75,0,0,100000-200000,0.00
1770,238910,bionic dues,2013-10-08,1,"Arcen Games, LLC","Arcen Games, LLC",windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading Cards;Steam Cloud,Indie;RPG;Strategy,Strategy;Rogue-like;Turn-Based,220,672,226,249,285,200000-500000,6.99
13111,586520,castle werewolf 3d,2017-02-07,1,Drop_Dead_Fred (Фред),Enjoy Games,windows,0,Single-player;Steam Trading Cards,Action;Indie,Action;Indie;Retro,0,39,33,274,317,50000-100000,3.19
12443,568090,tattletail,2016-12-28,1,Waygetter Electronics,"Little Flag Software, LLC",windows;mac,0,Single-player;Steam Achievements;Steam Trading Cards,Action;Indie,Horror;First-Person;Survival Horror,11,1335,157,148,191,100000-200000,3.99


### df2 -> df_cb [combined_features]

untuk Model 1: Content-Based Filtering (TF-IDF + cosine similarity), semua fitur label digabung untuk menghasilkan fitur utama yang akan digunakan

In [133]:
df_cb = df2.copy(deep=True)

# Gabungkan kolom-kolom teks
df_cb['combined_features'] = df_cb[['genres', 'steamspy_tags', 'categories', 'developer']].agg(';'.join, axis=1) # ; akan aturan pemisah

pd.set_option('display.max_colwidth', None) # Jangan potong isi kolom
pd.set_option('display.width', None) # Biarkan lebar menyesuaikan layar
pd.set_option('display.max_columns', None) # Tampilkan semua kolom jika banyak

# sederhanakan tabel
df_cb = df_cb[['name', 'combined_features']]
df_cb.sample(10)

,name,combined_features
26329,feditor,Free to Play;Indie;Simulation;Free to Play;Indie;Simulation;Single-player;Steam Leaderboards;Mark Craigie
1692,teleglitch: die more edition,Action;Indie;Rogue-like;Indie;Top-Down Shooter;Single-player;Steam Achievements;Full controller support;Steam Trading Cards;Steam Cloud;Test3 Projects
25841,nexvr video player,Utilities;Video Production;Utilities;Video Production;Partial Controller Support;NexVR
24275,lifespan 5seconds,Action;Indie;Action;Indie;Puzzle;Single-player;Full controller support;polytanQ
15260,party crashers,Indie;Racing;Racing;Indie;Multi-player;Local Multi-Player;Shared/Split Screen;Steam Achievements;Full controller support;Steam Cloud;Giant Margarita
5441,after all,Indie;RPG;RPG;Indie;RPGMaker;Single-player;JAMgames
24745,blast the past,Action;Casual;Free to Play;Indie;Simulation;Free to Play;VR;Indie;Single-player;The Work Club
19673,ball lab,Indie;Indie;Platformer;2D;Single-player;Steam Achievements;Partial Controller Support;Steam Cloud;Viktor Yurchuk
22662,dayd: through time. jurassic rush,Casual;Strategy;Casual;Strategy;Indie;Single-player;Steam Achievements;UniBit;Creobit
4624,inmind vr,Action;Casual;Free to Play;Indie;VR;Free to Play;Indie;Single-player;Full controller support;VR Support;Luden.io


- Setiap fitur penting digabung karena TF-IDF Butuh Representasi Teks Tunggal
- titik koma (**;**) akan digunakan kembali sebagai separator saat evaluasi

Selanjutnya adalah persiapam data mapping dan vector (dengan filter pembatas antara 20% - 80%)

In [134]:
# Buat mapping nama game ke index
indices = pd.Series(df_cb.index, index=df_cb['name'].str.lower()).drop_duplicates()

# TF-IDF Vectorizer dengan filtering kata umum/rare
tfidf = TfidfVectorizer(
    stop_words='english',
    max_df=0.8,     # Hapus kata yang muncul di >80% dokumen
    min_df=2        # Hapus kata yang muncul di <20% dokumen
)

# Transformasikan ke bentuk numerik
tfidf_matrix = tfidf.fit_transform(df_cb['combined_features'])

# Pastikan tidak ada NaN
assert not np.any(np.isnan(tfidf_matrix.toarray()))

# Ukuran matriks (baris: game, kolom: kata unik)
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

# Contoh kata-kata (fitur) yang dihasilkan
feature_names = tfidf.get_feature_names_out()
print(f"Contoh fitur: {feature_names[:10]}")

TF-IDF matrix shape: (27012, 6258)
Contoh fitur: ['06' '07th' '0cube' '100' '101' '10101' '10ants' '10th' '10tons' '11']


- Fitut di filter untuk mengurangi noise fitur yang terlalu sering/jarang muncul
- Didapatkan matriks TF-IDF dengan ukuran 27012 ✖ 6258

### df2 -> df_cf

Untuk model 2: Collaborative Filtering (SVD), Karena dataset tidak mengandung data eksplisit pengguna seperti user_id, maka untuk menggunakan metode ini, perlu membuat pseudo-user berdasarkan kombinasi genre/tag, lalu menggunakan rasio positif terhadap total ulasan (positif + negatif) sebagai proksi untuk rating.

In [135]:
df_cf = df2.copy(deep=True)

# Buat pseudo-user_id dari kombinasi genre
df_cf['user_id'] = df_cf['genres'].str.replace(';', '_').str.lower()

df_cf['rating'] = df_cf['positive_ratings'] / (df_cf['positive_ratings'] + df_cf['negative_ratings'] + 1e-5)
df_cf['rating'] = df_cf['rating'] * 10  # Skala ke 0-10

# Reader: skala rating
reader = Reader(rating_scale=(0, 10))

# Gunakan kolom yang sudah disiapkan
data = Dataset.load_from_df(df_cf[['user_id', 'name', 'rating']], reader)

# Bagi data latih dan uji
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
df_cf = df_cf[['user_id','name','rating']]
display(df_cf.sample(5))
display(df_cf.describe())

,user_id,name,rating
19652,casual,missing road,9.999950
8514,adventure_indie_rpg,candice debébé's incredibly trick lifestyle,8.308823
5154,indie,return null - episode 1,6.477272
19209,gore_adventure_casual_indie_simulation,the mysterious ship,9.999967
18231,indie,"burn, clown, burn!",4.285708


,rating
count,27012.000000
mean,7.146759
std,2.334223
min,0.000000
25%,5.833331
50%,7.605676
75%,8.939567
max,9.999999


Didapatkan dataset yang cocok untuk model CF, berupa matrix user - game - rating, dan rating punya skala yang sesuai (0-10)

## Modelling

### CBF

Karena ram tidak sanggup menjalankan proses *cosine similarity* dengan data 27k data, maka pakai sample 1000 game pertama. model akan menilai kemiripan `combined_features`.

In [136]:
# Ambil 10000 game pertama
tfidf_sample = tfidf_matrix[:10000]
cosine_sim = cosine_similarity(tfidf_sample)
cosine_sim

array([[1.        , 1.        , 0.8256112 , ..., 0.        , 0.13668096,
        0.        ],
       [1.        , 1.        , 0.8256112 , ..., 0.        , 0.13668096,
        0.        ],
       [0.8256112 , 0.8256112 , 1.        , ..., 0.        , 0.04050281,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.07742777,
        0.11245106],
       [0.13668096, 0.13668096, 0.04050281, ..., 0.07742777, 1.        ,
        0.20310715],
       [0.        , 0.        , 0.        , ..., 0.11245106, 0.20310715,
        1.        ]])

### CF
Dengan trainset yang sudah disiapkan matriks (user, game, rating), metode Singular Value Decomposition dapat dilakukan.

In [137]:
# Model SVD
algo = SVD()
algo.fit(trainset)

## Evaluation

### CBF

CBF menggunakan konten (genre). Jadi kita bisa mengukur precision dengan menghitung berapa banyak rekomendasi yang memiliki genre serupa dengan game yang sedang dianalisis.

In [138]:
def get_content_recommendations(title, cosine_sim, df, indices, top_k=10):
    idx = indices[title.lower()]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_k+1]
    game_indices = [i[0] for i in sim_scores]
    return df.iloc[game_indices], [i[1] for i in sim_scores]

def precision_at_k_cbf(title, recommended_df, df, k=10):
    true_combined_features = set(df[df['name'].str.lower() == title.lower()]['combined_features'].values[0].split(';'))
    rec_combined_features = recommended_df['combined_features'].apply(lambda x: set(x.split(';')))
    relevan_count = rec_combined_features.apply(lambda x: len(true_combined_features & x) > 2).sum()
    return relevan_count / k

def mean_cosine_similarity(sim_scores):
    return np.mean(sim_scores)

# Ambil 10 judul random dari 10.000 data pertama
titles = df2.iloc[:10000]['name'].dropna().sample(10, random_state=42).str.lower().tolist()
precision_scores = []
mean_cosine_scores = []

for title in titles:
    recommended_df, sim_scores = get_content_recommendations(title, cosine_sim, df_cb, indices)

    precision_cbf = precision_at_k_cbf(title, recommended_df, df_cb, k=10)
    precision_scores.append(precision_cbf)

    mean_sim = mean_cosine_similarity(sim_scores)
    mean_cosine_scores.append(mean_sim)

    print(f"Precision@10 (CBF) ({title}): {precision_cbf:.2f}")
    print(f"Mean Cosine Similarity ({title}): {mean_sim:.2f}\n")

# Hitung rata-rata precision dan mean cosine similarity
mean_precision = np.mean(precision_scores)
mean_cosine = np.mean(mean_cosine_scores)

print(f"Rata-rata Precision@10 (CBF): {mean_precision:.2f}")
print(f"Rata-rata Mean Cosine Similarity: {mean_cosine:.2f}")

Precision@10 (CBF) (affected zone tactics): 0.10
Mean Cosine Similarity (affected zone tactics): 0.48

Precision@10 (CBF) (lightning returns™: final fantasy® xiii): 0.50
Mean Cosine Similarity (lightning returns™: final fantasy® xiii): 0.39

Precision@10 (CBF) (hexodius): 1.00
Mean Cosine Similarity (hexodius): 0.71

Precision@10 (CBF) (quarries of scred): 1.00
Mean Cosine Similarity (quarries of scred): 0.85

Precision@10 (CBF) (banzai pecan: the last hope for the young century): 1.00
Mean Cosine Similarity (banzai pecan: the last hope for the young century): 0.54

Precision@10 (CBF) (maximum override): 0.80
Mean Cosine Similarity (maximum override): 0.52

Precision@10 (CBF) (nancy drew®: ransom of the seven ships): 1.00
Mean Cosine Similarity (nancy drew®: ransom of the seven ships): 0.80

Precision@10 (CBF) (sunrider: liberation day - captain's edition): 1.00
Mean Cosine Similarity (sunrider: liberation day - captain's edition): 0.62

Precision@10 (CBF) (hogs of war): 0.00
Mean Cosi

**Hasil:**
- `Precision@10: 0.74` artinya dari 10 game yang direkomendasikan memiliki fitur yang 74% relevan (setidaknya 3 fitur sama) dari rata-rata 10 sample random.
- `Mean Cosine Similarity: 0.61` Rata-rata kemiripan konten (TF-IDF) antara game input dan 10 rekomendasinya adalah 61% dari rata-rata 10 sample random.

### CF

In [139]:
# Evaluasi RMSE
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 2.3330


2.333034260136554

Rata-rata kesalahan prediksi model hanya sekitar 2.33 poin atau 23% (skala 0-10) terhadap rating aslinya.


CF memberikan prediksi rating. Maka Precision@k dihitung dari berapa banyak item yang direkomendasikan memiliki rating aktual di atas ambang batas, misalnya rating ≥ 4.

In [ ]:
def precision_at_k_cf(predictions, k=10, threshold=4.0):

    # Kelompokkan prediksi berdasarkan user
    top_k = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_k[uid].append((iid, est))

    # Ambil top-k prediksi tertinggi untuk setiap user
    for uid, user_ratings in top_k.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_k[uid] = user_ratings[:k]

    # Hitung Precision@k
    precisions = []
    for uid, user_ratings in top_k.items():
        n_relevant = sum((est >= threshold) for (_, est) in user_ratings)
        precisions.append(n_relevant / k)

    return sum(precisions) / len(precisions)

# Hitung precision@10
precision_cf = precision_at_k_cf(predictions)
print(f"Precision@10 (CF): {precision_cf:.2f}")

Precision@10 (CF): 0.29


## Deployment (Test Inferensi)

In [141]:
# CBF
def recommend_game(game_title, n=5):
    game_title = game_title.lower()

    if game_title not in indices:
        return f"Game '{game_title}' tidak ditemukan dalam dataset."

    idx = indices[game_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]

    recommended_games = [df.iloc[i[0]]['name'] for i in sim_scores]
    print(f"\nTop 5 rekomendasi game serupa '{game_title}':")
    return recommended_games

display(recommend_game('Dota 2'))

# CF
def recommend_for_user(user_id, algo, df_games, top_n=5):
    played = df_cf[df_cf['user_id'] == user_id]['name'].tolist()
    all_games = df_games['name'].unique()
    unseen = [g for g in all_games if g not in played]

    predictions = [(game, algo.predict(user_id, game).est) for game in unseen]
    top_recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]

    print(f"\nTop {top_n} rekomendasi game yang cocok dengan user '{user_id}':")
    return pd.DataFrame(top_recommendations, columns=['Game', 'Predicted Rating'])

recommend_for_user('action_indie', algo, df2)


Top 5 rekomendasi game serupa 'dota 2':


['Soup: the Game',
 'Kittypocalypse',
 'Counter-Strike: Global Offensive',
 'Team Fortress 2',
 'Unturned']


Top 5 rekomendasi game yang cocok dengan user 'action_indie':


,Game,Predicted Rating
0,fractured state,10.000000
1,pikuniku,10.000000
2,the coma: recut,9.938052
3,deranged rabbits,9.767697
4,logistical: russia,9.755017
